# Comparison of volumina of materials across materials databases

In [ ]:
from madas import MaterialsDatabase, Material
from madas.apis.api_core import APIClass, api_call

In [ ]:
from ase import Atoms

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import requests

In [ ]:
from typing import List

In [ ]:
import os

## Define APIs

### AFLOWlib

In [ ]:
class AFLOWlibAPI(APIClass):
    
    def __init__(self):
        pass
    
    @api_call
    def get_calculations_by_search(self, query_url: str) -> List[Material]:
        res = requests.get(query_url)
        materials = []
        for comp in res.json():
            mid = comp["auid"]
            mat_query = "/".join(["https://aflowlib.duke.edu", comp["aurl"].split(":")[1], "aflowlib.json"])
            mat_res = requests.get(mat_query).json()
            at = Atoms(cell=mat_res.pop("geometry"), 
                       scaled_positions=mat_res.pop("positions_fractional"), 
                       symbols=mat_res.pop("species"), 
                       pbc = True)
            materials.append(Material(mid, atoms=at, data = mat_res))
        return materials

### OQMD

In [ ]:
class OQMDAPI(APIClass):
    
    def __init__(self):
        pass
    
    @api_call
    def get_calculations_by_search(self, query_url: str) -> List[Material]:
        res = requests.get(query_url)
        materials = []
        for comp in res.json()["data"]:
            mid = f"OQMD:{comp.pop('entry_id')}"
            symbols, positions = [], []
            sites = comp.pop("sites")
            for site in sites:
                symbols.append(site.split("@")[0].strip())
                positions.append([float(x) for x in site.split("@")[1].strip().split(' ')])
            at = Atoms(cell=comp.pop("unit_cell"), 
                       scaled_positions=positions, 
                       symbols=symbols, 
                       pbc = True)
            materials.append(Material(mid, atoms=at, data = comp))
        return materials

### Materials Project

In [ ]:
from mp_api.client import MPRester

In [ ]:
class MPAPI(APIClass):
    
    def __init__(self, api_key=os.environ.get("MP_API_KEY", None)):
        self.mprester = MPRester(api_key=api_key)
    
    @api_call
    def get_calculations_by_search(self, **kwargs) -> List[Material]:
        materials = []
        with self.mprester as mpr:
            res = mpr.materials.summary.search(**kwargs)
            for mat in res:
                mid = mat.material_id.string
                materials.append(Material(mid, atoms=mat.structure.to_ase_atoms(), data=mat.dict()))
        return materials

## Initialize Databases

In [ ]:
db_aflow = MaterialsDatabase(filename = "NaCl_AFLOW.db", api = AFLOWlibAPI(), name="NaCl_AFLOW")

In [ ]:
db_oqmd = MaterialsDatabase(filename = "NaCl_OQMD.db", api = OQMDAPI(), name="NaCl_OQMD")

In [ ]:
db_mp = MaterialsDatabase(filename = "NaCl_MP.db", api = MPAPI(), name="NaCl_MP")

## Define queries

In [ ]:
AFLOW_query = r"https://aflowlib.org/API/aflux/?species(Na,Cl),$catalog(ICSD),$nspecies(2),$paging(1,1000)"

In [ ]:
oqmd_base_url = "http://oqmd.org/oqmdapi/formationenergy"

In [ ]:
def extend_url(base, field, separator="&") -> str:
    return f"{base}{separator}{field}"

In [ ]:
oqmd_query_url = extend_url(oqmd_base_url, "fields=name,entry_id,spacegroup,ntypes,band_gap,delta_e,unit_cell,configuration_label,sites", separator="?")
oqmd_query_url = extend_url(oqmd_query_url, "composition=Na1Cl1")

In [ ]:
mp_query={"elements":["Na", "Cl"], "num_elements":2}

## Download data 

In [ ]:
db_aflow.fill_database(AFLOW_query)

In [ ]:
db_oqmd.fill_database(oqmd_query_url)

In [ ]:
db_mp.fill_database(**mp_query)

## Compare entries

To analyse the spacegroup we make use of the ASE spacegroup module:

In [ ]:
from ase.spacegroup import get_spacegroup

Find spacegroups that are represented in all databases:

In [ ]:
all_spacegroups=set(get_spacegroup(entry.atoms).no for entry in db_aflow)

for db in [db_mp, db_oqmd]:
    sgs_in_db = set(get_spacegroup(entry.atoms).no for entry in db)        
    all_spacegroups.intersection_update(sgs_in_db)
    
print(f"Spacegroups available in all databases: {all_spacegroups}")    

In [ ]:
# filter and format data for plot
plot_data = []
materials = {sgn : [] for sgn in all_spacegroups}
for db in [db_aflow, db_mp, db_oqmd]:
    db_name = db.name.strip("NaCl_")
    for entry in db:
        sg = get_spacegroup(entry.atoms)
        # filter to add only materials that are present in all databases and have 2 atomic unit cells
        if sg.no in all_spacegroups and len(entry.atoms) == 2:
            plot_data.append([sg.symbol.replace(" ", ""), entry.atoms.get_volume(), db_name])
            materials[sg.no].append(entry)

In [ ]:
plt.figure(figsize = (6,5))
for db in [db_aflow, db_mp, db_oqmd]:
    db_name = db.name.strip("NaCl_")
    sgs, volumes = [], []
    for data in filter(lambda x: x[2] == db_name, plot_data):
        sgs.append(data[0])
        volumes.append(data[1])
    plt.scatter(sgs, volumes, label = db_name, marker="X")
plt.ylabel("Volume [Å$^3$]")
plt.xlabel("Space group")
plt.ylim(41,46.5)
plt.xlim(-0.5, 1.5)
plt.legend(fontsize = 25, fancybox = False, frameon = True, edgecolor="k")
# Uncomment this line to save the figure
#plt.savefig("./ComparingWebDBs.svg", format="svg", dpi=50)
plt.show()

In [ ]:
unique_sg_symbols = tuple(set(x[0] for x in plot_data))

In [ ]:
volumes_sg_1 = [x[1] for x in filter(lambda x: x[0]==unique_sg_symbols[0], plot_data)]
print(f"Maximal difference between volumes is {max(volumes_sg_1) - min(volumes_sg_1): .3f} for SG {unique_sg_symbols[0]}.")

In [ ]:
volumes_sg_2 = [x[1] for x in filter(lambda x: x[0]==unique_sg_symbols[1], plot_data)]
print(f"Maximal difference between volumes is {max(volumes_sg_2) - min(volumes_sg_2): .3f} for SG {unique_sg_symbols[1]}.")

## Verify structure equivalence

In [ ]:
from itertools import combinations
from ase.utils.structure_comparator import SymmetryEquivalenceCheck

check = SymmetryEquivalenceCheck(scale_volume=True)

# for each space group individually
for key, mats in materials.items():
    # for all combitnations of materials in that spacegroup
    for m1, m2 in combinations(mats, 2):
        # if they are not equivalent
        if not check.compare(m1.atoms, m2.atoms):
            # print their IDs
            print(m1.mid, m2.mid)